In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=211a60b4c7b23ee460e87a1750d37c6fcfa721572d6bd665bf3534efa22c8669
  Stored in directory: /home/onyxia/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [7]:
!pip install pandas transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.6 MB/s eta 0:00:0000:010:01


In [3]:
import os
import json
import numpy as np

# Définir le dossier contenant les fichiers JSONL à modifier
dataset_folders = [
    "OpenLLMText_Human"
]

# Fonction pour ajouter les nouvelles métadonnées au champ "extra"
def enrich_data(file_path, text_source):
    updated_data = []
    
    # Charger le fichier JSONL ligne par ligne
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    num_samples = len(lines)  # Nombre d'exemples dans le fichier

    # Générer les nouvelles métadonnées
    enriched_metadata = {
        "text_source": [text_source] * num_samples,
        "time_spent_per_word": np.random.uniform(0.1, 0.5, num_samples),
        "num_deletions": np.random.randint(5, 20, num_samples),
        "num_rewrites": np.random.randint(3, 15, num_samples),
        "copy_paste_usage": np.random.uniform(0, 0.3, num_samples),
        "pauses": np.random.uniform(1, 10, num_samples),
        "sentence_reordering": np.random.uniform(0.1, 0.5, num_samples),
    }

    # Modifier chaque ligne du fichier JSONL
    for i, line in enumerate(lines):
        data = json.loads(line.strip())  # Convertir en dict
        data["extra"].update({  # Ajouter les nouvelles informations
            "text_source": enriched_metadata["text_source"][i],
            "time_spent_per_word": float(enriched_metadata["time_spent_per_word"][i]),
            "num_deletions": int(enriched_metadata["num_deletions"][i]),
            "num_rewrites": int(enriched_metadata["num_rewrites"][i]),
            "copy_paste_usage": float(enriched_metadata["copy_paste_usage"][i]),
            "pauses": float(enriched_metadata["pauses"][i]),
            "sentence_reordering": float(enriched_metadata["sentence_reordering"][i])
        })
        updated_data.append(data)

    # Sauvegarder le fichier modifié
    new_file_path = file_path.replace(".jsonl", "_enriched.jsonl")
    with open(new_file_path, "w", encoding="utf-8") as f:
        for entry in updated_data:
            f.write(json.dumps(entry) + "\n")

    print(f"Fichier enrichi sauvegardé : {new_file_path}")

# Appliquer la transformation à tous les fichiers JSONL
for folder in dataset_folders:
    for filename in os.listdir(folder):
        if filename.endswith(".jsonl"):
            file_path = os.path.join(folder, filename)
            enrich_data(file_path, text_source=folder.replace("OpenLLMText_", ""))  # Déduire la source du texte


Fichier enrichi sauvegardé : OpenLLMText_Human/valid-dirty_enriched.jsonl
Fichier enrichi sauvegardé : OpenLLMText_Human/valid-dirty_enriched_enriched.jsonl
Fichier enrichi sauvegardé : OpenLLMText_Human/train-dirty_enriched.jsonl
Fichier enrichi sauvegardé : OpenLLMText_Human/train-dirty_enriched_enriched.jsonl
Fichier enrichi sauvegardé : OpenLLMText_Human/test-dirty_enriched_enriched.jsonl
Fichier enrichi sauvegardé : OpenLLMText_Human/test-dirty_enriched.jsonl


In [8]:
import os
import json
import pandas as pd

# Dossiers contenant les fichiers enrichis
dataset_folders = {
    "Human": "OpenLLMText_Human",
    "ChatGPT": "OpenLLMText_ChatGPT",
}

data = []

# Charger uniquement Human et ChatGPT
for source, folder in dataset_folders.items():
    for filename in os.listdir(folder):
        if filename.endswith("_enriched.jsonl"):
            file_path = os.path.join(folder, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                for line in f:
                    entry = json.loads(line.strip())
                    text = entry["text"]
                    extra = entry["extra"]
                    
                    # Ajouter toutes les informations du processus d'écriture
                    data.append({
                        "text": text,
                        "label": 0 if source == "Human" else 1,  # 0 = Humain, 1 = IA
                        "time_spent_per_word": extra["time_spent_per_word"],
                        "num_deletions": extra["num_deletions"],
                        "num_rewrites": extra["num_rewrites"],
                        "copy_paste_usage": extra["copy_paste_usage"],
                        "pauses": extra["pauses"],
                        "sentence_reordering": extra["sentence_reordering"],
                    })

# Convertir en DataFrame
df = pd.DataFrame(data)
print(df.head())  # Vérification des données


                                                text  label  \
0  Wednesday, April 6th, 2016\n\n"It is shameful ...      0   
1  SAN FRANCISCO (BCN)— A civil lawsuit filed Wed...      0   
2  Automated wheel changer. Image: Rio Tinto.\n\n...      0   
3  A Washington Post reporter admitted that Presi...      0   
4  An a-scientific paper, poor contribution of NG...      0   

   time_spent_per_word  num_deletions  num_rewrites  copy_paste_usage  \
0             0.142871             14            10          0.199902   
1             0.488851              8            10          0.268167   
2             0.260543             14            13          0.088837   
3             0.107372             13             3          0.140421   
4             0.161164              8            10          0.197570   

     pauses  sentence_reordering  
0  8.300962             0.182648  
1  5.216166             0.303016  
2  8.797998             0.145323  
3  7.013374             0.384291  
4  1.39

In [12]:
import os
import json
import pandas as pd

# Dossiers contenant les fichiers enrichis
dataset_folders = {
    "Human": "OpenLLMText_Human",
    "ChatGPT": "OpenLLMText_ChatGPT",
}

data = []

# Charger uniquement Human et ChatGPT
for source, folder in dataset_folders.items():
    for filename in os.listdir(folder):
        if filename.endswith("_enriched.jsonl"):
            file_path = os.path.join(folder, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                for line in f:
                    entry = json.loads(line.strip())
                    text = entry["text"]
                    extra = entry["extra"]

                    # Créer un dictionnaire avec le texte, le label et les métadonnées séparées
                    data.append({
                        "text": text,
                        "label": 0 if source == "Human" else 1,  # 0 = Humain, 1 = IA
                        "meta_time_spent_per_word": extra["time_spent_per_word"],
                        "meta_num_deletions": extra["num_deletions"],
                        "meta_num_rewrites": extra["num_rewrites"],
                        "meta_copy_paste_usage": extra["copy_paste_usage"],
                        "meta_pauses": extra["pauses"],
                        "meta_sentence_reordering": extra["sentence_reordering"],
                    })

# Convertir en DataFrame
df = pd.DataFrame(data)

# Séparer les textes et les métadonnées
df_meta = df.filter(like="meta_").astype(float)  # Extraire uniquement les colonnes méta

print("✅ Données chargées avec séparation des métadonnées")
print(df.head())  # Vérifier le DataFrame complet
print(df_meta.head())  # Vérifier les métadonnées seules


✅ Données chargées avec séparation des métadonnées
                                                text  label  \
0  Wednesday, April 6th, 2016\n\n"It is shameful ...      0   
1  SAN FRANCISCO (BCN)— A civil lawsuit filed Wed...      0   
2  Automated wheel changer. Image: Rio Tinto.\n\n...      0   
3  A Washington Post reporter admitted that Presi...      0   
4  An a-scientific paper, poor contribution of NG...      0   

   meta_time_spent_per_word  meta_num_deletions  meta_num_rewrites  \
0                  0.142871                  14                 10   
1                  0.488851                   8                 10   
2                  0.260543                  14                 13   
3                  0.107372                  13                  3   
4                  0.161164                   8                 10   

   meta_copy_paste_usage  meta_pauses  meta_sentence_reordering  
0               0.199902     8.300962                  0.182648  
1               0

In [ ]:
from transformers import BertTokenizer

# Charger un tokenizer pré-entraîné
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenisation rapide en batch
tokens = tokenizer.batch_encode_plus(
    df["text"].tolist(),  # Liste des textes
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
)

# Ajouter les tokens au DataFrame
df["input_ids"] = list(tokens["input_ids"])
df["attention_mask"] = list(tokens["attention_mask"])

print("✅ Tokenisation terminée.")



In [9]:
from transformers import BertTokenizer

# Charger un tokenizer pré-entraîné
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokeniser tous les textes et ajouter les métadonnées
df["tokens"] = df["text"].apply(lambda x: tokenizer(x, truncation=True, padding="max_length", max_length=512, return_tensors="pt"))

# Séparer les métadonnées numériques
meta_features = ["time_spent_per_word", "num_deletions", "num_rewrites", "copy_paste_usage", "pauses", "sentence_reordering"]
df_meta = df[meta_features].astype(float)

# Vérification
print(df["tokens"].iloc[0])
print(df_meta.head())


KeyboardInterrupt: 

In [1]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 26.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.2 MB/s eta 0:00:00
